# Assembling

This notebook is used to assemble a processable h5ad object for subsequent notebooks.

## Fill in input data, output and settings

In [ ]:
####################### TEST NAME ###################################
test = 'Aorta'
################### DEFINING STRATEGY ###############################
from_h5ad = True #option 1
assembling_10_velocity = False #option 2
assembling_10_public = False #option 3
convert_seurat = False #option 4

###################### INPUT DATA ###################################

#For option 1: The path to an existing .h5ad file
h5ad_path = '/home/rstudio/preprocessing/data/anndata/cropped_146.h5ad'
#################### Calc QC Columns ###############################
n_threads = 8

fragments_file = '/home/rstudio/preprocessing/data/bamfiles/fragments_cropped_146.bed'
promoters_gtf = '/home/rstudio/preprocessing/data/homo_sapiens.104.promoters2000.gtf'


##################### OUTPUT DATA ###################################
output_dir = '/home/rstudio/processed_data'

## Import modules

In [ ]:
# sctoolbox modules 
import sctoolbox.atac_tree as sub_tree
import sctoolbox.creators as cr
import sctoolbox.fragment_length as fragments
import sctoolbox.calc_overlap_pct as overlap
# import episcanpy
import episcanpy as epi

## Setup path handling object 

In [ ]:
# make an instance of the class
tree = sub_tree.ATAC_tree()
# set processing/output directory
tree.processing_dir = output_dir
# set sample/experiment.. 
tree.run = test

## Read in data

### Option 1: Read from .h5ad

In [ ]:
if from_h5ad:
    
    adata = epi.read_h5ad(h5ad_path)
    
    #Add information to the infoprocess
    cr.build_infor(adata, "Input_for_assembling", h5ad_path)
    cr.build_infor(adata, "Strategy", "Read from h5ad")

## Inspect adata

In [ ]:
display(adata)

In [ ]:
display(adata.var)

In [ ]:
display(adata.obs)

## Check for QC related columns

### 1. Check if mean fragment length column exists


In [ ]:
mfl_column = fragments.check_mfl(adata)
mfl_column

### 2. Check if percentage of reads in promoters column exists

In [ ]:
pct_rip_column = overlap.check_pct_fragments_in_promoters(adata)
pct_rip_column

## Calculate missing columns

### 1. calc mean fragment length if missing 

In [ ]:
if not mfl_column:
    adata = fragments.add_mfl_fragment(fragments_file, adata, n_threads)
adata.obs

### 2. Promotor enrichment 

In [ ]:
if not pct_rip_column:
    overlap.pct_fragments_in_promoters(adata, promoters_gtf, fragments_file, cb_col=None, nproc=1)
adata.obs

## Inspect adata.obs

In [ ]:
adata.obs

## Save adata to .h5ad

In [ ]:
adata_output = tree.assembled_anndata
adata_output

In [ ]:
#Saving the data
cr.build_infor(adata, "Test_number", test)
cr.build_infor(adata, "Anndata_path", output_dir)

adata_output = tree.assembled_anndata
adata.write(filename=adata_output)